# MATH 441 Group 5 Project

**Aziz, Mika, Spock**

# Project Title: Scheduling Employee to Shifts for a Bank

## 1) Introduction

**Problem Statement:**

How can we assign employees to suitable shifts in a financial institution to minimize labor costs while considering employee skill levels and salary?

**Relevant Real-world Examples:**

* Study how employee is assigned to shifts in banks or financial institutions.
* Explore existing optimization algorithms applied to employee scheduling problems.

**Data and Computations:**

 Data:
* Employee skill level.
* Employee training level.
* Employee salary.

**Note**: In search of a suitable dataset to apply this optimization on, we will attempt to find relevant employee scheduling information from local organizations like banks, investment firms and other financial institutions. There are of course some difficulties when it comes to obtaining said data:
1. Companies might not release such data publicly
2. Needed data might not be formatted as expected
3. A lack of data in general on employee scheduling

We will attempt to remedy this through a few methods, such as reaching out to companies to obtain anonymous data, generate data based on a few known paramater distributions etc.


But before we can start with the Data generation and LP solving procedures, we need to start by importing a few important libraries and modules, and performing any first-time setup necessary for the project.

## 1.5) Setting Up the Environment

First, install any required libraries.

In [ ]:
! pip3 install -r requirements.txt

Now, let's import our common libraries.

In [ ]:
import numpy as np
import random
import csv
import pulp
import pandas as pd 

Finally, let's define and assign values where needed to any important constants and variables that are used globally within the project

In [ ]:
employees_data = []
num_employees = 10
num_customer = 5

skill_levels = {"entry-level": 0, "junior": 1, "senior": 2, "manager": 3}

tasks_with_min_levels = {
    "Account opening": 0,
    "Credit card application": 0,
    "Loan Application": 1,
    "Mortgage Consultation": 2,
    "Retirement planning": 2,
    "Financial advising": 2,
    "Wealth management": 3
}

distribution = {
    "entry-level": 0.4,
    "junior": 0.3,
    "senior": 0.2,
    "manager": 0.1
}

salary_weight = {
    "K": 1000,
    "a": 3,
    "b": 5,
    "c": 10,
    "d": 35,
}

days = {"Monday": 0, 
        "Tuesday": 1, 
        "Wednesday": 2, 
        "Thursday": 3,
        "Friday": 4
}

demand_distribution = {
    "Account opening": 0.2,
    "Credit card application": 0.15,
    "Loan Application": 0.2,
    "Mortgage Consultation": 0.1,
    "Retirement planning": 0.2,
    "Financial advising": 0.1,
    "Wealth management": 0.05
}

day_label = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']

shift_label = ['Account opening', 'Credit card application', 'Loan Application', 'Mortgage Consultation', 
               'Retirement planning', 'Financial advising', 'Wealth management']

enumerate_shift = {
    'Account opening': 0, 
    'Credit card application': 1, 
    'Loan Application': 2, 
    'Mortgage Consultation': 3, 
    'Retirement planning': 4, 
    'Financial advising': 5, 
    'Wealth management': 6
}

In [ ]:
# This is a function that will print the assignment of the employees for shifts at each weekdays
def result(x, num_employees, day_label, shift_label):
    df = pd.DataFrame(index=range(5), columns=range(7)).fillna('')
    df = df.astype(str)

    for i in range(num_employees):
        for j in range(7):
            for d in range(5):
                if x[(i, j, d)].varValue == 1:
                    df.at[d, j] += (',' if df.at[d, j] else '') + str(i+1)

    df = df.fillna(' ')
    df = df.rename(index=dict(zip(range(5), day_label)), columns=dict(zip(range(7), shift_label)))
    
    return df

## 2) Defining the parameters and variables of the scheduling problem

Before we begin with the sovling of the integer programming problem at the core of our project, we need to first define our parameters and variables. The variables that we identify also need to be enumerated so we can use a python solver.

**Parameter for the shifts**

- Shift $j$: There are seven types of shifts, denoted as $1 \leq j \leq m$. There is a skill requirement for the employees to do each shift.
    - 1 - Account opening - entry 
    - 2 - Credit card application - entry 
    - 3 - Loan Application - junior 
    - 4 - Mortgage Consultation - senior 
    - 5 - retirement planning - senior
    - 6 - financial advising - senior
    - 7 - wealth management - manager
- Weekdays $w$: We will assume that all bank employees only work five days a week, so the days are enumerated to $0 \leq w \leq 4$
    - 0 - Monday
    - 1 - Tuesday
    - 2 - Wednesday
    - 3 - Thursday
    - 4 - Friday
- Demand $d_{jw}$: The demand for each weekdays are different for each type of shifts. We will use a random distribution for the demand. This will be an array of length 5, representing each weekdays, with a subarray of length 7, representing different type of shifts. For example: 
    - [[1,2,3,0,1,5,2], [0,0,2,3,4,3,2], [3,3,2,3,4,3,2], [5,5,3,4,5,3,2], [2,3,4,5,6,5,4]]: This shows that on Monday, there needs to be one employee assigned to account opening, two employee assigned to credit card application, and so on...

**Parameter for employees**

- Employee $i$: The employee id will be enumerated to $1 \leq i \leq n$. The employee data will also consist of their skill levels, training level, and salary.
- Skill level $l$: The skills will be divided into four levels, $0 \leq l\leq 3$, which also represent what shift they are authorized to do.
    - 0 - entry-level
    - 1 - junior
    - 2 - senior
    - 3 - manager
- Training level $t_{ij}$: This is an array of length n, the number of employees, with subarrays of length 7, the type of shifts. This represents the training level of each employee for different types of shift. The subarray contain seven values from 1 to 100. If the employee doesn't have the required skill to do the shift, the value will be set to 0. For example: 
    - Entry-level employee: [32,15,0,0,0,0,0]
    - Senior level employee: [45,72,61,13,4,80,0]
- Salary $s$: The salary of each employee is related to the training level. For entry-level employees who can only do shift one and two, their salary will be less if they are less trained. 
$$ 
s = k + a \sum_i (t_{i,1} + t_{i,2}) + b \sum_i t_{i,3} + c \sum_i (t_{i,4} + t_{i,5} + t_{i,6}) + d \sum_i t_{i,7}
$$
$$
\text{where $k,a,b,c,d$ are all real numbers, with $k$ as the base salary and $a,b,c,d$ as the weight on each type of shifts paid.}
$$

**Decision variables:**<br>
- Let $x_{ijd} \in {0,1}$ be a binary variable, where $x_{ijd} = 1$ if employee $i$ is assigned to shift $j$ at day $d$, and 0 otherwise.

## 3) Data

**Employee data**

In [ ]:
def generate_employee_data_custom_distribution(num_employees, skill_levels, tasks_with_min_levels, salary_weight, distribution):

    # Calculate the number of employees in each skill level based on the distribution
    num_employees_distribution = {level: int(pct * num_employees) for level, pct in distribution.items()}
    
    # Adjust for any rounding differences to ensure the total count matches num_employees
    while sum(num_employees_distribution.values()) < num_employees:
        num_employees_distribution[random.choice(list(num_employees_distribution.keys()))] += 1
        
    # Generate data for each employee based on the distribution
    for skill_level_label, count in num_employees_distribution.items():
        skill_level = skill_levels[skill_level_label]
        for _ in range(count):
            training_level_array = []
            for task, min_level in tasks_with_min_levels.items():
                if skill_level >= min_level:
                    training_level = random.randint(1, 100)  # Training levels range from 1 to 100
                else:
                    training_level = 0
                training_level_array.append(training_level)
            salary = salary_weight["K"] + salary_weight["a"] * (training_level_array[0]+training_level_array[1]) \
            + salary_weight["b"] * (training_level_array[2]) \
            + salary_weight["c"] * (training_level_array[3]+training_level_array[4]+training_level_array[5]) \
            + salary_weight["d"] * (training_level_array[6])
            employees_data.append((skill_level_label, training_level_array, salary))
    
    # Shuffle the data to mix skill levels
    random.shuffle(employees_data)
    
    return employees_data

employees_data = generate_employee_data_custom_distribution(num_employees, skill_levels, tasks_with_min_levels, salary_weight, distribution)

# Output the first and last five employees
first_five = employees_data[:5]
last_five = employees_data[-5:]

print("First five employees:")
for i, (skill, training_level, salary) in enumerate(first_five):
    print(f"Employee {i + 1}: Skill Level - {skill}, Training level - {training_level}, Salary - {salary}")

print("\nLast five employees:")
for i, (skill, training_level, salary) in enumerate(last_five):
    print(f"Employee {len(employees_data) - 4 + i}: Skill Level - {skill}, Training level - {training_level}, Salary - {salary}")

# Save to CSV file
csv_file_path = "employees_data.csv"
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Skill Level", "Training level", "Salary"])
    for skill, training_level, salary in employees_data:
        writer.writerow([skill, training_level, salary])

**Customer demand data**

For each day, we need to decide how many customers demand for an appointment. Banks usually data from previous years to readily predict how many customers need employees with shift j in a day. Since we couldn't find any data related, we will generate a random number of customers for each shift depending on the distribution. We decided to distribute the shift j by 0.2, 0.15, 0.2, 0.1, 0.2, 0.1, 0.05 respectively. The distribution was created as demand_distribution in part 1.5. 

In [ ]:
def generate_customer_data_custom_distribution(max, days, distribution):
    customer_data = []

    for day, index in days.items():
        # we would have a different number of customers demanded for each day from 0 to num_customer
        num_customer = np.random.randint(0, max)

        # calculate the number of customer in shifts based on the distribution
        num_customer_distribution = {shift: int(pct * num_customer) for shift, pct in distribution.items()}
        
        # Adjust for any rounding differences to ensure the total count matches num_employees
        while sum(num_customer_distribution.values()) < num_customer:
            num_customer_distribution[random.choice(list(num_customer_distribution.keys()))] += 1

        # for each shift, get the distribution
        for shift_label, count in num_customer_distribution.items():
            customer_data.append((day, shift_label, count))
    
    return customer_data

customer_data = generate_customer_data_custom_distribution(num_customer, days, demand_distribution)

# Output the first and last five customer
first_five = customer_data[:5]
last_five = customer_data[-5:]

print("First five shift demand:")
for i, (day, shift, shift_demand) in enumerate(first_five):
    print(f"Shift {i+1}: Day - {day}, Shift - {shift}, Shift Demand - {shift_demand}")

print("\nLast five shift demand:")
for i, (day, shift, shift_demand) in enumerate(last_five):
    print(f"Shift {i+1}: Day - {day}, Shift - {shift}, Shift Demand - {shift_demand}")

# Save to CSV file
csv_file_path = "customer_data.csv"
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Day", "Shift", "Shift Demand"])
    for day, shift, shift_demand in customer_data:
        writer.writerow([day, shift, shift_demand])

**Visualization of how many employees need to be assigned to a shift on each weekday**

In [ ]:
# enumerate the data, so it's easier to use
customer = []

shift = {
    "Account opening": 0,
    "Credit card application": 1,
    "Loan Application": 2,
    "Mortgage Consultation": 3,
    "Retirement planning": 4,
    "Financial advising": 5,
    "Wealth management": 6
}

for data in customer_data:
    # change days string to integer
    day = days[data[0]]
    # change shift string to integer
    shift_type = shift[data[1]]
    demand = data[2]
    customer.append([day, shift_type, demand])

In [ ]:
# rows as days and columns as shift types
demand_df = pd.DataFrame()

for day, shift, demand in customer:
    demand_df.at[day, shift] = demand

day_label = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
shift_label = ['Account opening', 'Credit card application', 'Loan Application', 'Mortgage Consultation', 
               'Retirement planning', 'Financial advising', 'Wealth management']

demand_df = demand_df.rename(index = dict(zip(range(5), day_label)), columns= dict(zip(range(7), shift_label)))
demand_df 

## 4) Formulate the problem as a linear programming problem

To summarize our problem, each shift per day has a customer demand that we need to fulfill. The amount of customer demand determines how much employees we assign to the shift in each day and there are 7 different types of shift. We will assume that each employee can only be assign to at most one type of shift per day.

Next, we need to assign shifts to employees in such a way that for each shift, we select employees with the highest training level for that shift, then choose an employee with lowest salary among the selected employees to assign this employee to the shift, aiming to minimize overall labor costs.

**Objective:**<br>
Minimize the total labor cost, which is the sum of the salaries of the assigned employees:
$$
\sum_i \sum_j \sum_d x_{ijd} \times \text{s$_i$}
$$
$$
\text{where s$_i$ is the salary of employee $i$.}
$$

**Constraints:**

1) An employee can be assigned to at most one shift for each day:
$$
\sum_i \sum_d x_{ijd}  \leq  1 \ , \ \ \text{for all shift} \ j
$$

2) Ensure that only employees with the required skill level can be assigned to shifts:
$$
x_{ijd} = 0 \ , \ \ \text{if employee $i$ does not have the required skull level for shift $j$}
$$

Now, incorporating the additional requirement of selecting the employee with the highest training level and lowest salary for each shift, we add the following constraints:

3) For each shift, select the employee with the highest training level:
$$
\sum_d \sum_i t_{ij}\times x_{ijd} = \max{\{t_{i'j}| i'\text{ has the required skill level for shift } j\}}
$$
$$
\text{where $t_{ij}$ is the training level of employee $i$ for shift $j$.}
$$

4) Ensure that the selected employee has the lowest salary among those with the highest training level for each shift:
$$
\sum_d \sum_i s_{i}\times x_{ijd} = \min{\{s_{i'}| i'\text{ has the highest training level for shift } j\}}
$$
$$
\text{where $s_{i}$ is the salary of employee $i$.}
$$

5) Each shifts has the required number of employees at day d
$$\sum_i x_{ijd} = d_{jd}, \ \ \text{for all shifts} \ j, \ \ \text{for all days} \ d$$



## 5) Python Implementation Using Pulp

In [ ]:
salaries = [employee[2] for employee in employees_data]
skill_prob = pulp.LpProblem("Shift_Assignment", pulp.LpMinimize)
x = pulp.LpVariable.dicts("x", [(i, j, d) for i in range(num_employees) for j in range(7) for d in range(5)], cat="Binary")
skill_prob += pulp.lpSum(x[i, j, d] * salaries[i] for i in range(num_employees) for j in range(7) for d in range(5))

# get the demand at day d for shift j
d_jw = np.zeros((5,7))
for day, shift, demand in customer:
    d_jw[day, shift] = demand

# An employee can be assigned to at most one shift at day d
for i in range(num_employees):
    for d in range(5):
        skill_prob += pulp.lpSum(x[i, j, d] for j in range(7)) <= 1

# Each shifts has the required number of employees at day d
for d in range(5):
    for j in range(7):
        skill_prob += pulp.lpSum(x[i, j, d] for i in range(num_employees)) == d_jw[d, j]

### New Constraint:

## introduce new variable
# minimum skill level for each pair weekday d and shift type j
z  = {}
for day, shift, demand in customer_data:
    z[(enumerate_shift[shift])] = tasks_with_min_levels[shift]

# Constraint: Ensure that only employees with the required skill level can be assigned to shifts
for i in range(num_employees):
    for j in range(7):
       for d in range(5):
            # if skill level of employee i is not as high as skill required to do shift j
            if skill_levels[employees_data[i][0]] < z[(j)]:
                # don't assign employee i to shift j at any day d
                skill_prob += x[i, j, d] == 0

skill_prob.solve()
print("Status:", pulp.LpStatus[skill_prob.status])

In [ ]:
result(x, num_employees, day_label, shift_label)

### Implementation of restricted training level

In [ ]:
### Previous set-up:
prob = pulp.LpProblem("Shift_Assignment", pulp.LpMinimize)
x = pulp.LpVariable.dicts("x", [(i, j, d) for i in range(num_employees) for j in range(7) for d in range(5)], cat="Binary")
prob += pulp.lpSum(x[i, j, d] * salaries[i] for i in range(num_employees) for j in range(7) for d in range(5))

# get the demand at day d for shift j
d_jw = np.zeros((5,7))
for day, shift, demand in customer:
    d_jw[day, shift] = demand

# An employee can be assigned to at most one shift at day d
for i in range(num_employees):
    for d in range(5):
        prob += pulp.lpSum(x[i, j, d] for j in range(7)) <= 1

# Each shifts has the required number of employees at day d
for d in range(5):
    for j in range(7):
        prob += pulp.lpSum(x[i, j, d] for i in range(num_employees)) == d_jw[d, j]

# Constraint: Ensure that only employees with the required skill level can be assigned to shifts
for i in range(num_employees):
    for j in range(7):
       for d in range(5):
            # if skill level of employee i is not as high as skill required to do shift j
            if skill_levels[employees_data[i][0]] < z[(j)]:
                # don't assign employee i to shift j at any day d
                skill_prob += x[i, j, d] == 0

### New Constraints

# Constraint: For each shift, select the employee with the highest training level
for j in range(7):
        prob += pulp.lpSum(skill_levels[employees_data[i][0]] * x[i, j, d] for i in range(num_employees)) == 
        max(training_levels[i][j] for i in range(num_employees) if skill_levels[i] >= required_skill_levels[j] for d in range(5))

# Constraint: Ensure that the selected employee has the lowest salary among those with the highest training level
for j in range(7):
    prob += pulp.lpSum(salaries[i] * x[i, j, d] for i in range(num_employees)) == 
    min(salaries[i] for i in range(num_employees) if training_levels[i][j] == max(training_levels[i][j] 
    for i in range(num_employees) if skill_levels[i] >= required_skill_levels[j]) for d in range(5))

prob.solve()
print("Status:", pulp.LpStatus[prob.status])

## 6) Solution

In [ ]:
solution_df = result(x, num_employees, day_label, shift_label)